# Computing in the Small
## Week 4 Binary Numbers
Decimal numbers are written as a combination  of the digits $0-9$. To count up, we increment the lowest digit by adding $1$ until we hit $9$ in which case we wrap around to $0$ and increment the next higher digit.  Carries can occur as a chain reaction, e.g., when adding $1$ to $999$.

In binary or base $2$, the same thing applies except that the only possible digits are $0$ and $1$.  If we add $1$ and $1$ we get $0$ in the lowest place and then carry a $1$ resulting in $10$.

### Binary Numbers in Python
A binary number is just another way of expressing a number.  To distinguish the base, Python uses the convention that binary numbers are written startng with "0b".  In fact there are three special bases recognized by Python.  All of the following represent the number 15:
* binary (base $2$) numbers, e.g., 0b1111
* octal (base $8$) numbers, e.g., 0o17
* hex (or hexadecimal, base $16$) numbers, e.g., 0xf
In the case of binary we use $0-1$, and octal uses $0-7$ but hex needs sixteen different "digits" and therefore in addition to $0-9$ it treats the letters "a-f" as digits.

In [ ]:
for i in range(25):
    print(f"decimal {i} is binary {bin(i)} octal {oct(i)} and hex {hex(i)}")

Octal is mainly added for historical purposes as it was used in really old computers.  Hexadecimal, on the other hand, is used extensively.  We can easily convert from hex of octal to and from binary by groupng the digits (binary digits are called "bits") in groups of 3 (for octal) or 4 (hex).

Most computer languages support integer values with a fixed number of bits with typical lengths including $8$ ($8$ bits is called a byte), $16$, $32$ and sometimes $64$ or $128$.  These larger groupings are called a *WORD* and in some cases twice that space is called a double word, *DWORD*

Python,as we have seen allows really large numbers (subject to available memory) and therefore there is no fixed bound on the number of digits.

In [ ]:
print(bin(777777777))

### Modeling Binary Numbers as Lists
To better understand how counting in binary works, we can code our own version.  We will represent numbers as a list of bits.  We will store the bits backwards because as the number grows, it is easier to add digits to the end of a list than to the beginning, however we will account for this when we print the list as a number.

To increment the number we need to add one to the lowest bit and then carry if necessary.  To handle the general case, we make `increment` call a more general `inc` function which specifies which bit to increment.  The carry might take us out of the range of the digits we have so far, in which case we append a new $1$ to the end of our list (when we print the number it will appear on the left).

In [ ]:
def increment(number):
    return inc(number, 0)



def print_number(vector):
    print("".join(map(str, reversed(vector))))



In [ ]:
def inc(number, bit):
    if bit >= len(number):
        number += [0] * (bit - len(number) + 1)
        number[bit] = 1
        return number
    if number[bit] == 0:
        number[bit] = 1
        return number
    else:
        number[bit] = 0
        return inc(number, bit + 1)

Since the lists have the bits in the reverse order of how we usually read them, we can define a helper function to print the list as a normal-looking number.  The follwoing program turns each of the numbers in our list into a string and then joins them together.  The "" indicates that we want nothing separating them.

In [ ]:
def print_number(number):
    print("".join(map(str, reversed(number))))

Now we can test our program by starting at our representation for $0$, `[0]`, incrementing it several times and using the `print_number` function to see what we get.

In [ ]:
number = [0]
for i in range(10):
    print_number(number)
    number = increment(number)

Some problems for anyone wanting to stretch are to continue our implementation of numbers as lists to:
1. Write a function to add two numbers in this form.  Hint, use the `inc` function on one number every time a $1$ occurs in the other.  It doesnot matter which number you choose to apply to which and the order of incrementing the bits also does not matter.
1. Write a function to convert a normal integer into a number list.
1. Write a function that will reurn the normal integer represented by one of these lists.

### Bitwise Operations
Unlike incrementing, bitwise operations combine numbers one bit at a time with no carry.  We have already seen an example of this when we considered adding bits modulo 2 or as it is written in Python:
```
sum = (x + y) % 2
```
The supported bitwise operations are:
* and & (1 if and only if both arguments are 1)
* or |  (1 if either arguments is 1 or if both are)
* xor ^ (1 if the arguments differ...actually this is the same operation as addition mod 2)
Note this is why Pything writes powers as `x**y` unlike other languages that use `x^y` as Python was saving `^` to mean something different. 

Each of these operators has a different typical use in programming.

The `and` operator gives the ability to restrict to a subset of the bits in a number.  One argument is called the "mask".  Often these masks have the form of a fixed number of bits.

For example if the `mask = 0b111` (= 7) and the number to be tested is `0b110110` (= 54), the result of anding the two is the last three bits of the test number, `0b110`.

Note that the `and` operation is commutative and thus there is no distinction between the mask and the test values except convention.  Typically we code the mask as a constant and the number being tested comes from outside.

In [ ]:
mask = 0b111
test = 0b110110
print(bin(mask & test))

### Logical Shifts
Python supports moving all the bits left or right one place.

#### Left Shift <<
A left shift is equivalent to adding a trailing zero.  For base $10$ numbers, adding a $0$ multiplies the number by $10$ and for base $2$ numbers adding a $0$ multiplies a number by $2$.  Here we start with $1$ and shift left shift a few times.  The expression:
```
x <<= 1
```
is simply a short way to write:
```
x = x << 1
```
and this convention holds for most Python operators (arithmetic as well as bitwise).  NOte also that:
```
x <<= 1
```
has the same effect as multiplying by 2:
```
x *= 2
```
but on almost all computers is much faster.

In [ ]:
x = 1
for i in range(5):
    print(f"dec({x}) --> bin({x:b})")
    x <<= 1

#### Right Shift >>
Right shift similarly has the effect of dividing by $2$ but without remainder.  Recall we had two division operators, $/$ which produced floating point numbers and $//$ with truncation, and:
```
x >>= 1
```
has the same effect dividing by 2 wih truncation:
```
x //= 2
```
but the first version is **much** faster on most computers.

In [ ]:
x = 36
for i in range(7):
    print(f"dec({x}) --> bin({x:b})")
    x >>= 1

### Pascal Revisited
Last week we used a function to calculate rows of Pascal's triangle and then reduce them mod 2, and we prduced some elegant looking pictures.  As we mentioned last week, element $k$ in row $n$ of Pascal's triangle represents the number of ways that we can choose $k$ objects from a tota set of $n$.  We write this number as 
$$n \choose k$$
So what we want to know is the value of:
$${n \choose k} \mod 2$$
and there is a theorem by [Lucas](https://en.wikipedia.org/wiki/Lucas%27s_theorem) which implies that what we need is to write both $n%$ and $k$ in binary in which case the answer is $1$ if and only if every $1$ in the binary expansion of $k$ lines up with a $1$ in the binary expansion of $n$ or in other words if:
```
n | k == n
```


In [ ]:
def pascal_mod2(n):
  return [1 if n == (i | n) else 0 for i in range(n + 1)]

In [ ]:
from timeit import timeit

timeit('pascal_mod2(100_000)', globals=globals(), number=10)

The `number` argument represents how many times to run to get an average time.  Note that this program can be made roughly twice as fast if we also add in the optimzation that as we have noticed last week one can get the second half of the row by mirroring the first half.  On my laptop, row one hundred thousand took about 1/20 of a second to compute.

In [ ]:
def pascal_image_symmetric(n):
    img = []
    for i in range(n):
        img.append(((n - i - 1)//2) * [0] + pascal_mod2(i) + ((n - i)//2)* [0])
    return img

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.imshow(pascal_image_symmetric(1_000))